**Task 1: Pre-process the dataset**

All the imports:

In [1]:
import csv
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
import datetime 

**Check how many patients/variables there are**

In [14]:
df = pd.read_csv("dataset_mood_smartphone.csv")
counted = df.groupby(["id"]).size()
print (len(counted))
print (counted)

27
id
AS14.01    21999
AS14.02    14581
AS14.03    14425
AS14.05    15745
AS14.06    18092
AS14.07    16045
AS14.08     7902
AS14.09    10886
AS14.12    17311
AS14.13    19592
AS14.14     9286
AS14.15     2848
AS14.16     3982
AS14.17    15826
AS14.19    11397
AS14.20     3620
AS14.23    21852
AS14.24    14430
AS14.25    12589
AS14.26    16403
AS14.27    14575
AS14.28    19276
AS14.29    17499
AS14.30    17279
AS14.31    11889
AS14.32    11193
AS14.33    16390
dtype: int64


In [15]:
counted = df.groupby(["variable"]).size()
print (len(counted))
print (counted)

19
variable
activity                22965
appCat.builtin          91288
appCat.communication    74276
appCat.entertainment    27125
appCat.finance            939
appCat.game               813
appCat.office            5642
appCat.other             7650
appCat.social           19145
appCat.travel            2846
appCat.unknown            939
appCat.utilities         2487
appCat.weather            255
call                     5239
circumplex.arousal       5643
circumplex.valence       5643
mood                     5641
screen                  96578
sms                      1798
dtype: int64


There are __***27 patients***__, names from 01 to 33 (some numbers are missing)

There are __***19 variables***__



**Making individual files for each patient:**

In [2]:
RawFile = "dataset_mood_smartphone.csv"

with open(RawFile) as infile:
    reader = csv.reader(infile)
    #input_header = reader.next()
    for row in reader:
        patient = row[1]
        CleanedFile = "./data/" + patient +".csv"
        with open(CleanedFile, 'a', newline='') as outfile:
            writer = csv.writer(outfile)
            if row[4] != "NA": #deleting rows with NA as VALUE
                writer.writerow(row[2:5])

With good old-fashioned CTRL-F method we discover that: (AND REALIZING YOU DIDN'T NEED TO CHECK THAT CAUSE ITS ALL IN THE ASSIGNMENT DESCRIPTION)

activity                Numerical value - use mean

appCat.builtin          Time - add times

appCat.communication    Time - add times

appCat.entertainment    Time - add times

appCat.finance          Time - add times

appCat.game             Time - add times

appCat.office           Time - add times

appCat.other            Time - add times

appCat.social           Time - add times

appCat.travel           Time - add times

appCat.unknown          Time - add times

appCat.utilities        Time - add times

appCat.weather          Time - add times

call                    Numerical value - add

circumplex.arousal      Numerical value - use mean

circumplex.valence      Numerical value - use mean

mood                    Numerical value - use mean

screen                  Time

SMS                     Numerical values - add



In [2]:
df = pd.read_csv("./test_scripts/AS14.01.csv")
counted = df.groupby(["date"]).size()
print (len(counted))
print (counted)


72
date
2014-02-17      2
2014-02-18      1
2014-02-19      9
2014-02-20      5
2014-02-21      1
2014-02-22      3
2014-02-25      3
2014-02-26     15
2014-02-27      9
2014-02-28      4
2014-03-01      1
2014-03-03      1
2014-03-05      1
2014-03-06      1
2014-03-07      5
2014-03-10      1
2014-03-11      3
2014-03-12      8
2014-03-13      4
2014-03-14      4
2014-03-15      1
2014-03-16      3
2014-03-17      1
2014-03-18      3
2014-03-19      1
2014-03-20    113
2014-03-21    632
2014-03-22    381
2014-03-23    368
2014-03-24    685
             ... 
2014-04-06    355
2014-04-07    467
2014-04-08    678
2014-04-09    665
2014-04-10    387
2014-04-11    474
2014-04-12    234
2014-04-13    191
2014-04-14    569
2014-04-15    659
2014-04-16    553
2014-04-17    406
2014-04-18    385
2014-04-19    363
2014-04-20    414
2014-04-21    280
2014-04-22    732
2014-04-23    538
2014-04-24    570
2014-04-25    520
2014-04-26    312
2014-04-27    248
2014-04-28    392
2014-04-29    412
20

In [10]:
df.dtypes

date         object
time         object
variable     object
value       float64
dtype: object

In [20]:
df = pd.read_csv("./test_scripts/AS14.01.csv")
fsdkjfhsdf = df.drop(['time'], axis=1)

testoutpath = "./data/summary/AS14.01.csv"
fsdkjfhsdf.to_csv(testoutpath, encoding='utf-8', index=False)

Want to creat a table/csv that would summarize each day. Then we can proceed with another algorithm.

Produce a day by day summary of the data
---

In [9]:
variables= ['mood', 'circumplex.arousal', 'circumplex.valence', 'activity', 'screen', 'call', 'sms', 'appCat.builtin', 'appCat.communication', 'appCat.entertainment', 'appCat.finance', 'appCat.game', 'appCat.office','appCat.other', 'appCat.social', 'appCat.travel', 'appCat.unknown', 'appCat.utilities', 'appCat.weather']
patient = ['01', '02','03','05','06','07','08','09','12','13','14','15','16','17','19','20','23','24','25','26','27','28','29','30','31','32','33']
for patientnumber in patient:
    inpath = "./data/AS14."+patientnumber+".csv"
    outpath = "./data/summary/AS14." + patientnumber + ".csv"

    df = pd.read_csv(inpath)

    df["time"] = pd.to_datetime(df["time"]) 
    #df['date_minus_time'] = df["time"].apply( lambda df : datetime.datetime(year=df.year, month=df.month, day=df.day))
    #df.set_index(df["date_minus_time"],inplace=True) # using date as index 
    df.index = df['time']
    df = df.sort_index()


    byday = [] #variables summmed/meaned for each day
    for i, j in enumerate(variables):
        #print(i, j)
        dfvariables = df.loc[df['variable'] == j]
        if i < 4:
            data = dfvariables['value'].resample('D').mean()
    #         print(i, j, len(data))
    #         print (len(data))
        else:
            data = dfvariables['value'].resample('D').sum()
            #print(i, j, len(data))
            #print (len(data))
        byday.append(data)

    daysSet = pd.concat(byday, axis=1)
    daysSet.columns = variables
    daysSet= daysSet.replace(0, np.nan)

    #print(daysSet)
    daysSet.to_csv(outpath, encoding='utf-8', index=True)

DELETE/FILL IN MISSING DATA
---

In [31]:
patientnumber = "01"
inpath = "./data/summary/AS14." + patientnumber + ".csv"
outpath = "./data/summary/AS14." + patientnumber + "SSSSSSS.csv"
df = pd.read_csv(inpath)

#df.dtypes
#print (df) #cheched datatype of each column

#df = df.dropna(subset=["mood"], inplace=True) #drops the rows with empty mood
        
df.dropna(thresh=4, inplace=True)     # drop the rows with at least 4 LEGIT values 
                                            # GOOD TO START WITH CAUSE THERE IS NO POINT IN FILLING MOOD DATA 
                                            # WHEN THERE IS NO OTHER DATA IN THAT ROW
                                            # MIGHT NEED TO CHANGE TO 3

# Most of this is 0 values: 
# game
# office
# weather 
# Probably more accurate to fill it in with 0

# Rest - median 

# s=df.isnull().cumsum()
# t1=df[(s==1).shift(-1).fillna(False)].stack().reset_index(level=0,drop=True)
# t2=df.lookup(s.idxmax()+1,s.idxmax().index)
# df.fillna(t1/(2**s)+t2*(1-0.5**s)*2/2)

#df.mood.fillna(method='bfill',inplace=True)  #for back fill - THIS IS CURRENTLY DONE IN THE DATA - This is good for MOOD
#df.mood.fillna(method='ffill',inplace=True)  #for forward-fill - THIS IS CURRENTLY DONE IN THE DATA - This is good for MOOD

#df.fillna({'game':0, 'office':0, 'weather': 0}, inplace=True) #fill in the empty rows "call" and "sms" with 0


#df.fillna(df.mood.mean(),inplace=True)       # use mean of the data to fill missind data - this not not good for MOOD (too much difference)

#df.mood.fillna(df.mood.median(),inplace=True)     # use median to till in missing data - this not not good for MOOD

#df = df.fillna(0) #fill in the empty rows with 0
#df.fillna({'game':0, 'office':0, 'weather': 0}, inplace=True) #fill in the empty rows "call" and "sms" with 0


print (df)
#df.to_csv(outpath, encoding='utf-8', index=False)


KeyError: 78